# Cybersecurity Data Science Project - Data Preprocessing
## Dataset Collection and Preprocessing Notebook

This notebook provides code to download and preprocess three datasets:
1. **SEC EDGAR Company Facts** - Financial/Organizational Metrics
2. **VERIS Community Database (VCDB)** - Cybersecurity Incidents/Breaches
3. **BuiltWith API** - Company Technology Stack Data

---

## Installation Requirements

Run this cell first to install required packages:

In [1]:
!pip install requests pandas numpy matplotlib seaborn tqdm

## Core Imports

In [2]:
# Standard library imports
import os
import json
import zipfile
import glob
import time
import warnings
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional

# Data manipulation
import pandas as pd
import numpy as np

# HTTP requests
import requests
from urllib.parse import urlencode

# Progress bars
from tqdm import tqdm

# Visualization (optional, for EDA)
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

print("✅ All imports successful!")

✅ All imports successful!


## Configuration and Setup

In [3]:
# Create directories for data storage
DATA_DIR = Path('./data')
SEC_DIR = DATA_DIR / 'sec_edgar'
VCDB_DIR = DATA_DIR / 'vcdb'
BUILTWITH_DIR = DATA_DIR / 'builtwith'
OUTPUT_DIR = DATA_DIR / 'processed'

# Create all directories
for directory in [DATA_DIR, SEC_DIR, VCDB_DIR, BUILTWITH_DIR, OUTPUT_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

# SEC EDGAR Configuration
SEC_COMPANYFACTS_URL = "https://www.sec.gov/Archives/edgar/daily-index/xbrl/companyfacts.zip"
SEC_USER_AGENT = "Charles Morris cmorr107@charlotte.edu"  # SEC requires User-Agent for identification

# VCDB Configuration
VCDB_GITHUB_URL = "https://github.com/vz-risk/VCDB.git"
VCDB_DATA_PATH = VCDB_DIR / 'VCDB/data/json/validated'

# BuiltWith Configuration
BUILTWITH_API_KEY = "10e02b2f-da09-4bd3-9cbd-f3396da56301"  # BuiltWith API key configured
BUILTWITH_FREE_API_URL = "https://api.builtwith.com/free1/api.json"

print("✅ Configuration complete!")
print(f"Data directory: {DATA_DIR.absolute()}")

✅ Configuration complete!
Data directory: /content/data


---
# Dataset 1: SEC EDGAR Company Facts
## Financial and Organizational Metrics

### Step 1.1: Download SEC EDGAR Data

In [4]:
def download_sec_companyfacts():
    """
    Downloads the SEC EDGAR companyfacts.zip file.
    This contains JSON files for all public companies.
    """
    print("Downloading SEC EDGAR Company Facts...")
    print(f"Source: {SEC_COMPANYFACTS_URL}")

    zip_path = SEC_DIR / 'companyfacts.zip'

    # Check if already downloaded
    if zip_path.exists():
        print(f"✅ File already exists at {zip_path}")
        return zip_path

    # SEC requires User-Agent header
    headers = {
        'User-Agent': SEC_USER_AGENT
    }

    try:
        response = requests.get(SEC_COMPANYFACTS_URL, headers=headers, stream=True)
        response.raise_for_status()

        # Get file size for progress bar
        total_size = int(response.headers.get('content-length', 0))

        # Download with progress bar
        with open(zip_path, 'wb') as f:
            with tqdm(total=total_size, unit='B', unit_scale=True, desc='Downloading') as pbar:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
                    pbar.update(len(chunk))

        print(f"✅ Download complete: {zip_path}")
        return zip_path

    except requests.exceptions.RequestException as e:
        print(f"❌ Error downloading SEC data: {e}")
        return None

# Execute download
sec_zip_path = download_sec_companyfacts()

Source: https://www.sec.gov/Archives/edgar/daily-index/xbrl/companyfacts.zip


Downloading: 100%|██████████| 1.34G/1.34G [00:13<00:00, 98.4MB/s]

✅ Download complete: data/sec_edgar/companyfacts.zip


### Step 1.2: Extract SEC EDGAR Data

In [5]:
def extract_sec_companyfacts(zip_path):
    """
    Extracts the companyfacts.zip file.
    """
    if not zip_path or not zip_path.exists():
        print("❌ ZIP file not found")
        return None

    extract_path = SEC_DIR / 'companyfacts'

    # Check if already extracted
    if extract_path.exists() and list(extract_path.glob('*.json')):
        print(f"✅ Already extracted to {extract_path}")
        return extract_path

    print("Extracting SEC EDGAR Company Facts...")

    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)

        json_files = list(extract_path.glob('**/*.json'))
        print(f"✅ Extracted {len(json_files)} company JSON files to {extract_path}")
        return extract_path

    except Exception as e:
        print(f"❌ Error extracting: {e}")
        return None

# Execute extraction
sec_extract_path = extract_sec_companyfacts(sec_zip_path)

Extracting SEC EDGAR Company Facts...
✅ Extracted 19006 company JSON files to data/sec_edgar/companyfacts


### Step 1.3: Parse SEC JSON Files to DataFrame

In [6]:
def parse_sec_company_json(json_path):
    """
    Parses a single SEC company JSON file and extracts key financial metrics.
    """
    try:
        with open(json_path, 'r') as f:
            data = json.load(f)

        # Extract company metadata
        cik = data.get('cik')
        entity_name = data.get('entityName', '')

        # Extract facts
        facts = data.get('facts', {})

        # Common financial metrics to extract
        metrics_to_extract = {
            'Assets': 'us-gaap:Assets',
            'Liabilities': 'us-gaap:Liabilities',
            'Revenue': 'us-gaap:Revenues',
            'NetIncome': 'us-gaap:NetIncomeLoss',
            'CommonStockSharesOutstanding': 'us-gaap:CommonStockSharesOutstanding',
            'NumberOfEmployees': 'dei:EntityNumberOfEmployees',
        }

        company_data = {
            'CIK': cik,
            'EntityName': entity_name,
        }

        # Extract most recent value for each metric
        for metric_name, metric_key in metrics_to_extract.items():
            taxonomy, tag = metric_key.split(':')

            if taxonomy in facts and tag in facts[taxonomy]:
                metric_data = facts[taxonomy][tag]
                units = metric_data.get('units', {})

                # Get USD values if available
                if 'USD' in units and units['USD']:
                    # Get most recent filing
                    sorted_values = sorted(units['USD'], key=lambda x: x.get('end', ''), reverse=True)
                    if sorted_values:
                        company_data[metric_name] = sorted_values[0].get('val')
                        company_data[f'{metric_name}_Date'] = sorted_values[0].get('end')
                # For non-monetary values (like employee count)
                elif 'pure' in units and units['pure']:
                    sorted_values = sorted(units['pure'], key=lambda x: x.get('end', ''), reverse=True)
                    if sorted_values:
                        company_data[metric_name] = sorted_values[0].get('val')
                        company_data[f'{metric_name}_Date'] = sorted_values[0].get('end')

        return company_data

    except Exception as e:
        # print(f"Error parsing {json_path}: {e}")
        return None


def create_sec_dataframe(extract_path, max_companies=None):
    """
    Creates a pandas DataFrame from all SEC company JSON files.

    Args:
        extract_path: Path to extracted JSON files
        max_companies: Limit number of companies to process (for testing)
    """
    if not extract_path or not extract_path.exists():
        print("❌ Extract path not found")
        return None

    print("Parsing SEC company JSON files...")

    # Find all JSON files
    json_files = list(extract_path.glob('**/*.json'))

    if max_companies:
        json_files = json_files[:max_companies]

    print(f"Processing {len(json_files)} company files...")

    # Parse all files
    companies_data = []
    for json_path in tqdm(json_files, desc="Parsing companies"):
        company_data = parse_sec_company_json(json_path)
        if company_data:
            companies_data.append(company_data)

    # Create DataFrame
    df = pd.DataFrame(companies_data)

    print(f"\n✅ Created DataFrame with {len(df)} companies and {len(df.columns)} columns")
    print(f"\nColumns: {list(df.columns)}")

    return df


# Execute parsing (start with 100 companies for testing)
# Remove max_companies parameter to process all companies
sec_df = create_sec_dataframe(sec_extract_path, max_companies=100)

# Display sample
if sec_df is not None:
    display(sec_df.head())
    print(f"\nDataFrame shape: {sec_df.shape}")
    print(f"Missing values:\n{sec_df.isnull().sum()}")

Parsing SEC company JSON files...
Processing 100 company files...


Parsing companies: 100%|██████████| 100/100 [00:02<00:00, 44.44it/s]


✅ Created DataFrame with 100 companies and 10 columns

Columns: ['CIK', 'EntityName', 'Assets', 'Assets_Date', 'Liabilities', 'Liabilities_Date', 'NetIncome', 'NetIncome_Date', 'Revenue', 'Revenue_Date']


,CIK,EntityName,Assets,Assets_Date,Liabilities,Liabilities_Date,NetIncome,NetIncome_Date,Revenue,Revenue_Date
0,1118384,ROGERS INTERNATIONAL RAW MATERIALS FUND LP,4885232.0,2018-03-31,172417.0,2018-03-31,38244.0,2018-03-31,NaN,NaN
1,1865975,AfterNext HealthTech Acquisition Corp.,259516986.0,2023-06-30,11956415.0,2023-06-30,NaN,NaN,NaN,NaN
2,0001039001,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1271057,"CHINA-BIOTICS, INC",281197960.0,2013-06-30,56982665.0,2013-03-31,153920.0,2013-06-30,NaN,NaN
4,1826991,Trepont Acquisition Corp I,233516890.0,2022-03-31,13875883.0,2022-03-31,8235729.0,2022-03-31,NaN,NaN



DataFrame shape: (100, 10)
Missing values:
CIK                  1
EntityName           0
Assets              18
Assets_Date         18
Liabilities         25
Liabilities_Date    25
NetIncome           20
NetIncome_Date      20
Revenue             55
Revenue_Date        55
dtype: int64


### Step 1.4: Save SEC Data to CSV

In [7]:
if sec_df is not None:
    output_file = OUTPUT_DIR / 'sec_company_financials.csv'
    sec_df.to_csv(output_file, index=False)
    print(f"✅ Saved SEC data to: {output_file}")
    print(f"File size: {output_file.stat().st_size / 1024 / 1024:.2f} MB")

✅ Saved SEC data to: data/processed/sec_company_financials.csv
File size: 0.01 MB


---
# Dataset 2: VERIS Community Database (VCDB)
## Cybersecurity Incidents and Breaches

### Step 2.1: Clone VCDB Repository

In [8]:
def clone_vcdb_repo():
    """
    Clones the VCDB GitHub repository.
    """
    repo_path = VCDB_DIR / 'VCDB'

    # Check if already cloned
    if repo_path.exists() and (repo_path / '.git').exists():
        print(f"✅ VCDB repository already exists at {repo_path}")
        return repo_path

    print("Cloning VCDB repository...")
    print("This may take several minutes...")

    try:
        import subprocess
        result = subprocess.run(
            ['git', 'clone', VCDB_GITHUB_URL, str(repo_path)],
            capture_output=True,
            text=True
        )

        if result.returncode == 0:
            print(f"✅ Successfully cloned VCDB to {repo_path}")
            return repo_path
        else:
            print(f"❌ Error cloning repository: {result.stderr}")
            return None

    except FileNotFoundError:
        print("❌ Git is not installed. Please install git or manually download the repository.")
        print(f"Manual download: {VCDB_GITHUB_URL}")
        return None
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# Execute clone
vcdb_repo_path = clone_vcdb_repo()

Cloning VCDB repository...
This may take several minutes...
✅ Successfully cloned VCDB to data/vcdb/VCDB


### Step 2.2: Parse VCDB JSON Files

In [9]:
def parse_vcdb_incident(json_path):
    """
    Parses a single VCDB incident JSON file.
    """
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # Extract key fields
        incident_data = {
            'incident_id': data.get('incident_id', ''),
            'victim_name': data.get('victim', {}).get('victim_id', ''),
            'victim_industry': data.get('victim', {}).get('industry', ''),
            'victim_country': data.get('victim', {}).get('country', [''])[0] if data.get('victim', {}).get('country') else '',
            'victim_state': data.get('victim', {}).get('state', ''),
            'victim_employee_count': data.get('victim', {}).get('employee_count', ''),
            'victim_revenue': data.get('victim', {}).get('revenue', {}).get('iso_currency_code', ''),

            # Timeline
            'incident_year': data.get('timeline', {}).get('incident', {}).get('year', None),
            'discovery_date': data.get('timeline', {}).get('discovery', {}).get('unit', ''),

            # Action details
            'action_malware': 'Malware' in str(data.get('action', {})),
            'action_hacking': 'Hacking' in str(data.get('action', {})),
            'action_social': 'Social' in str(data.get('action', {})),
            'action_misuse': 'Misuse' in str(data.get('action', {})),
            'action_physical': 'Physical' in str(data.get('action', {})),
            'action_error': 'Error' in str(data.get('action', {})),
            'action_environmental': 'Environmental' in str(data.get('action', {})),

            # Actor
            'actor_external': 'External' in str(data.get('actor', {})),
            'actor_internal': 'Internal' in str(data.get('actor', {})),
            'actor_partner': 'Partner' in str(data.get('actor', {})),

            # Attributes
            'confidentiality_breach': data.get('attribute', {}).get('confidentiality', {}).get('data_disclosure', '') == 'Yes',
            'integrity_breach': 'integrity' in data.get('attribute', {}),
            'availability_breach': 'availability' in data.get('attribute', {}),

            # Impact
            'records_lost': data.get('attribute', {}).get('confidentiality', {}).get('data_total', None),

            # Source info
            'reference': data.get('reference', ''),
            'source_id': data.get('source_id', ''),
        }

        return incident_data

    except Exception as e:
        # print(f"Error parsing {json_path}: {e}")
        return None


def create_vcdb_dataframe(data_path=None):
    """
    Creates a pandas DataFrame from VCDB incident JSON files.
    """
    if data_path is None:
        data_path = VCDB_DATA_PATH

    if not data_path.exists():
        print(f"❌ VCDB data path not found: {data_path}")
        print("Please ensure VCDB repository is cloned correctly.")
        return None

    print(f"Searching for VCDB incident files in: {data_path}")

    # Find all JSON files
    json_files = list(data_path.glob('**/*.json'))

    print(f"Found {len(json_files)} incident files")
    print("Parsing incidents...")

    # Parse all files
    incidents_data = []
    for json_path in tqdm(json_files, desc="Parsing incidents"):
        incident_data = parse_vcdb_incident(json_path)
        if incident_data:
            incidents_data.append(incident_data)

    # Create DataFrame
    df = pd.DataFrame(incidents_data)

    print(f"\n✅ Created DataFrame with {len(df)} incidents and {len(df.columns)} columns")

    return df


# Execute parsing
vcdb_df = create_vcdb_dataframe()

# Display sample
if vcdb_df is not None:
    display(vcdb_df.head())
    print(f"\nDataFrame shape: {vcdb_df.shape}")
    print(f"\nIncidents by year:")
    print(vcdb_df['incident_year'].value_counts().sort_index())

Searching for VCDB incident files in: data/vcdb/VCDB/data/json/validated
Found 9911 incident files
Parsing incidents...


Parsing incidents: 100%|██████████| 9911/9911 [00:01<00:00, 8722.81it/s]



✅ Created DataFrame with 9911 incidents and 25 columns


,incident_id,victim_name,victim_industry,victim_country,victim_state,victim_employee_count,victim_revenue,incident_year,discovery_date,action_malware,action_hacking,action_social,action_misuse,action_physical,action_error,action_environmental,actor_external,actor_internal,actor_partner,confidentiality_breach,integrity_breach,availability_breach,records_lost,reference,source_id
0,DEDBEF2D-F59B-4098-ADC1-EF69F2FF96C0,Horizon Blue Cross Blue Shield,524114,US,NJ,1001 to 10000,,2013,,False,False,False,False,False,False,False,False,False,False,False,False,True,839711.0,http://newyork.cbslocal.com/2013/12/07/horizon...,vcdb
1,DFC900AD-AC45-4E59-A97E-EA60C60C0B97,Ministry of Defence,928110,GB,,Over 100000,,2004,,False,False,False,False,False,False,False,False,False,False,True,False,False,NaN,https://www.databreaches.net/uk-vicarious-liab...,vcdb
2,A73F6074-FD44-40BB-81B5-77B0482A2708,Danske Bank A/S,522110,DK,,10001 to 25000,,2016,Months,False,False,False,False,True,False,False,False,False,False,True,False,True,1.0,http://yle.fi/uutiset/danske_bank_employee_sus...,vcdb
3,3ED0B543-A40D-44A4-A2F8-6AEE63DD88BB,Limburg Police Department,921,NL,,Unknown,,2016,NA,False,False,False,False,False,False,False,False,False,False,True,False,False,NaN,http://www.nltimes.nl/2016/10/10/corrupt-limbu...,vcdb
4,9E67B9A1-2C34-4D9B-AD31-94D73D939D70,Murfreesboro Ambulance Service,621910,US,TN,Unknown,,2008,,False,False,False,False,True,False,False,False,False,False,True,True,False,NaN,http://www.murfreesboropost.com/medlocks-convi...,vcdb



DataFrame shape: (9911, 25)

Incidents by year:
incident_year
1971       1
1984       2
1994       1
1995       1
1998       1
1999       2
2000       1
2001       5
2002       6
2003      13
2004      17
2005      23
2006      23
2007      51
2008      82
2009      94
2010     588
2011     547
2012    1276
2013    1939
2014    1011
2015     924
2016     843
2017     603
2018     343
2019     333
2020     311
2021     117
2022       6
2023     747
Name: count, dtype: int64


### Step 2.3: Save VCDB Data to CSV

In [10]:
if vcdb_df is not None:
    output_file = OUTPUT_DIR / 'vcdb_cybersecurity_incidents.csv'
    vcdb_df.to_csv(output_file, index=False)
    print(f"✅ Saved VCDB data to: {output_file}")
    print(f"File size: {output_file.stat().st_size / 1024 / 1024:.2f} MB")

✅ Saved VCDB data to: data/processed/vcdb_cybersecurity_incidents.csv
File size: 2.98 MB


Dataset 3

In [21]:
# ============================================================================
# Dataset 3: Digital Asset Exposure (Derived from SEC Data)
# ============================================================================

# First, fetch SIC codes from SEC API if not present
def add_sic_from_sec_api(df):
    """
    Fetch SIC codes using SEC's submissions API.
    """
    print("Fetching SIC codes from SEC API...")

    headers = {'User-Agent': SEC_USER_AGENT}
    sic_codes = {}

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Fetching SIC codes"):
        cik = str(row['CIK']).zfill(10)  # Pad with zeros

        try:
            url = f"https://data.sec.gov/submissions/CIK{cik}.json"
            response = requests.get(url, headers=headers)

            if response.status_code == 200:
                data = response.json()
                sic = data.get('sic')
                if sic:
                    sic_codes[row['CIK']] = int(sic)

            time.sleep(0.11)  # Rate limiting - SEC allows 10 requests/second

        except Exception as e:
            continue

    df['SIC'] = df['CIK'].map(sic_codes)
    print(f"✅ Retrieved SIC codes for {df['SIC'].notna().sum()} companies\n")

    return df


# Check if SIC codes need to be fetched
if sec_df is not None and 'SIC' not in sec_df.columns:
    print("⚠️  SIC codes missing. Fetching from SEC API...")
    print("⏱️  This will take ~11 seconds for 100 companies (rate limited)\n")
    sec_df = add_sic_from_sec_api(sec_df)


def create_digital_exposure_features(df):
    """
    Create digital asset exposure proxy variables from SEC financial data.

    This approach uses company size, revenue, and industry classification
    as proxies for digital maturity and technology adoption.
    """
    import numpy as np

    df_enriched = df.copy()

    print("Creating digital exposure proxy features from SEC data...")
    print(f"📋 Available columns: {list(df.columns)}\n")

    # Check which columns are actually available
    has_employees = 'NumberOfEmployees' in df.columns
    has_revenue = 'Revenue' in df.columns
    has_assets = 'Assets' in df.columns
    has_sic = 'SIC' in df.columns

    # =======================================================================
    # Feature 1: Company Size Score (Employee Count or Assets)
    # =======================================================================
    if has_employees:
        employees = df_enriched['NumberOfEmployees'].fillna(0)
        try:
            df_enriched['size_score'] = pd.qcut(
                employees, q=5, labels=[0, 0.25, 0.5, 0.75, 1.0], duplicates='drop'
            ).astype(float)
        except:
            df_enriched['size_score'] = (employees - employees.min()) / (employees.max() - employees.min())
        print("✅ Using NumberOfEmployees for size score")
    elif has_assets:
        assets = df_enriched['Assets'].fillna(0)
        try:
            df_enriched['size_score'] = pd.qcut(
                assets, q=5, labels=[0, 0.25, 0.5, 0.75, 1.0], duplicates='drop'
            ).astype(float)
        except:
            df_enriched['size_score'] = (assets - assets.min()) / (assets.max() - assets.min())
        print("✅ Using Assets for size score")
    else:
        df_enriched['size_score'] = 0.5
        print("⚠️  No size metric available")

    # =======================================================================
    # Feature 2: Revenue Score
    # =======================================================================
    if has_revenue:
        revenue = df_enriched['Revenue'].fillna(0)
        try:
            df_enriched['revenue_score'] = pd.qcut(
                revenue, q=5, labels=[0, 0.25, 0.5, 0.75, 1.0], duplicates='drop'
            ).astype(float)
        except:
            df_enriched['revenue_score'] = (revenue - revenue.min()) / (revenue.max() - revenue.min())
        print("✅ Using Revenue for revenue score")
    elif has_assets:
        assets = df_enriched['Assets'].fillna(0)
        try:
            df_enriched['revenue_score'] = pd.qcut(
                assets, q=5, labels=[0, 0.25, 0.5, 0.75, 1.0], duplicates='drop'
            ).astype(float)
        except:
            df_enriched['revenue_score'] = (assets - assets.min()) / (assets.max() - assets.min())
        print("✅ Using Assets for revenue score")
    else:
        df_enriched['revenue_score'] = 0.5

    # =======================================================================
    # Feature 3: Industry-Based Scores (if SIC available)
    # =======================================================================
    if has_sic:
        # High-Tech Industry
        high_tech_sic = [3570, 3571, 3572, 3575, 3576, 3577, 3578, 7370, 7371, 7372,
                         7373, 7374, 7375, 7378, 7379, 4812, 4813, 4899, 5961, 5734,
                         3812, 3823, 3825, 3826, 3827, 3829, 3674]
        df_enriched['is_high_tech'] = df_enriched['SIC'].isin(high_tech_sic).astype(float)

        # Financial Services
        finance_sic = [6020, 6021, 6022, 6029, 6035, 6036, 6141, 6153, 6159,
                       6211, 6221, 6282, 6289, 6311, 6321, 6324, 6331, 6351, 6361, 6371, 6399]
        df_enriched['is_finance'] = df_enriched['SIC'].isin(finance_sic).astype(float)

        # Healthcare
        healthcare_sic = [8000, 8011, 8021, 8031, 8041, 8042, 8043, 8049,
                          8050, 8051, 8060, 8062, 8063, 8069, 8071, 8082, 8090, 8093, 8099]
        df_enriched['is_healthcare'] = df_enriched['SIC'].isin(healthcare_sic).astype(float)

        # Retail/E-commerce
        retail_sic = [5200, 5211, 5311, 5331, 5399, 5411, 5412, 5441, 5451, 5461,
                      5531, 5621, 5651, 5661, 5712, 5731, 5734, 5735, 5812, 5813,
                      5912, 5941, 5942, 5943, 5944, 5945, 5961, 5963, 5990, 5999]
        df_enriched['is_retail'] = df_enriched['SIC'].isin(retail_sic).astype(float)

        print("✅ Created industry indicators from SIC codes")
    else:
        df_enriched['is_high_tech'] = 0
        df_enriched['is_finance'] = 0
        df_enriched['is_healthcare'] = 0
        df_enriched['is_retail'] = 0
        print("⚠️  SIC codes not available, using Assets/Revenue only")

    # =======================================================================
    # Composite Score: Digital Exposure Index (0-1 scale)
    # =======================================================================
    df_enriched['digital_exposure_score'] = (
        0.20 * df_enriched['size_score'] +
        0.20 * df_enriched['revenue_score'] +
        0.30 * df_enriched['is_high_tech'] +
        0.15 * df_enriched['is_finance'] +
        0.10 * df_enriched['is_healthcare'] +
        0.05 * df_enriched['is_retail']
    )

    df_enriched['digital_exposure_score'] = df_enriched['digital_exposure_score'].clip(0, 1)

    # =======================================================================
    # Summary Statistics
    # =======================================================================
    print("\n" + "="*70)
    print("✅ Digital Exposure Features Created!")
    print("="*70)
    print(f"\n📊 Summary Statistics:")
    print(f"   Companies: {len(df_enriched)}")
    print(f"   Mean score: {df_enriched['digital_exposure_score'].mean():.3f}")
    print(f"   Median score: {df_enriched['digital_exposure_score'].median():.3f}")
    print(f"   Std dev: {df_enriched['digital_exposure_score'].std():.3f}")

    if has_sic:
        print(f"\n🏢 Industry Distribution:")
        print(f"   High-tech: {df_enriched['is_high_tech'].sum()} ({df_enriched['is_high_tech'].mean()*100:.1f}%)")
        print(f"   Financial: {df_enriched['is_finance'].sum()} ({df_enriched['is_finance'].mean()*100:.1f}%)")
        print(f"   Healthcare: {df_enriched['is_healthcare'].sum()} ({df_enriched['is_healthcare'].mean()*100:.1f}%)")
        print(f"   Retail: {df_enriched['is_retail'].sum()} ({df_enriched['is_retail'].mean()*100:.1f}%)")

    high_exposure = df_enriched[df_enriched['digital_exposure_score'] > 0.7]
    print(f"\n🎯 High Exposure (>0.7): {len(high_exposure)} ({len(high_exposure)/len(df_enriched)*100:.1f}%)")

    if len(high_exposure) > 0:
        print(f"\n   Top 5 by digital exposure:")
        top5 = df_enriched.nlargest(5, 'digital_exposure_score')[
            ['EntityName', 'digital_exposure_score', 'is_high_tech', 'is_finance']
        ]
        print(top5.to_string(index=False))

    print("="*70 + "\n")

    return df_enriched


# Execute
if sec_df is not None:
    sec_df_with_digital_exposure = create_digital_exposure_features(sec_df)

    # Save
    output_file = OUTPUT_DIR / 'sec_company_financials_with_digital_exposure.csv'
    sec_df_with_digital_exposure.to_csv(output_file, index=False)
    print(f"✅ Saved to: {output_file}")
    print(f"   Size: {output_file.stat().st_size / 1024 / 1024:.2f} MB\n")

    # Display sample
    print("📋 Sample:")
    display(sec_df_with_digital_exposure[
        ['EntityName', 'Assets', 'Revenue', 'is_high_tech', 'is_finance', 'digital_exposure_score']
    ].head(10))
else:
    print("❌ SEC data not available")

⚠️  SIC codes missing. Fetching from SEC API...
⏱️  This will take ~11 seconds for 100 companies (rate limited)

Fetching SIC codes from SEC API...


Fetching SIC codes: 100%|██████████| 100/100 [00:26<00:00,  3.77it/s]

✅ Retrieved SIC codes for 86 companies

Creating digital exposure proxy features from SEC data...
📋 Available columns: ['CIK', 'EntityName', 'Assets', 'Assets_Date', 'Liabilities', 'Liabilities_Date', 'NetIncome', 'NetIncome_Date', 'Revenue', 'Revenue_Date', 'SIC']

✅ Using Assets for size score
✅ Using Revenue for revenue score
✅ Created industry indicators from SIC codes

✅ Digital Exposure Features Created!

📊 Summary Statistics:
   Companies: 100
   Mean score: 0.154
   Median score: 0.101
   Std dev: 0.143

🏢 Industry Distribution:
   High-tech: 9.0 (9.0%)
   Financial: 13.0 (13.0%)
   Healthcare: 0.0 (0.0%)
   Retail: 1.0 (1.0%)

🎯 High Exposure (>0.7): 0 (0.0%)

✅ Saved to: data/processed/sec_company_financials_with_digital_exposure.csv
   Size: 0.02 MB

📋 Sample:


,EntityName,Assets,Revenue,is_high_tech,is_finance,digital_exposure_score
0,ROGERS INTERNATIONAL RAW MATERIALS FUND LP,4885232.0,NaN,0.0,1.0,0.201026
1,AfterNext HealthTech Acquisition Corp.,259516986.0,NaN,0.0,0.0,0.101026
2,,NaN,NaN,0.0,0.0,0.001026
3,"CHINA-BIOTICS, INC",281197960.0,NaN,0.0,0.0,0.101026
4,Trepont Acquisition Corp I,233516890.0,NaN,0.0,0.0,0.101026
5,Vodka Brands Corp,120869.0,NaN,0.0,0.0,0.051026
6,ONCOR ELECTRIC DELIVERY TRANSITION BOND CO LLC,81991000.0,NaN,0.0,0.0,0.101026
7,"DICE THERAPEUTICS, INC.",550673000.0,NaN,0.0,0.0,0.151026
8,ASIA PACIFIC WIRE & CABLE CORPORATION LIMITED,389384000.0,462265000.0,0.0,0.0,0.155400
9,Zymergen Inc.,470968000.0,7425000.0,0.0,0.0,0.151096


---
# Data Integration & Final Output

### Step 4.1: Summary of All Datasets

In [22]:
print("=" * 80)
print("DATASET SUMMARY")
print("=" * 80)

datasets_summary = []

# SEC EDGAR
if sec_df is not None:
    datasets_summary.append({
        'Dataset': 'SEC EDGAR Company Facts',
        'Rows': len(sec_df),
        'Columns': len(sec_df.columns),
        'Output File': 'sec_company_financials.csv',
        'Status': '✅ Complete'
    })
else:
    datasets_summary.append({
        'Dataset': 'SEC EDGAR Company Facts',
        'Status': '❌ Not processed'
    })

# VCDB
if vcdb_df is not None:
    datasets_summary.append({
        'Dataset': 'VCDB Cybersecurity Incidents',
        'Rows': len(vcdb_df),
        'Columns': len(vcdb_df.columns),
        'Output File': 'vcdb_cybersecurity_incidents.csv',
        'Status': '✅ Complete'
    })
else:
    datasets_summary.append({
        'Dataset': 'VCDB Cybersecurity Incidents',
        'Status': '❌ Not processed'
    })

# BuiltWith
if 'builtwith_df' in locals() and builtwith_df is not None:
    datasets_summary.append({
        'Dataset': 'BuiltWith Technology Stack',
        'Rows': len(builtwith_df),
        'Columns': len(builtwith_df.columns),
        'Output File': 'builtwith_technology_stack.csv',
        'Status': '✅ Complete'
    })
else:
    datasets_summary.append({
        'Dataset': 'BuiltWith Technology Stack',
        'Status': '⚠️ Requires API key and domain list'
    })

summary_df = pd.DataFrame(datasets_summary)
display(summary_df)

print(f"\n📁 All output files saved to: {OUTPUT_DIR.absolute()}")

DATASET SUMMARY


,Dataset,Rows,Columns,Output File,Status
0,SEC EDGAR Company Facts,100,11,sec_company_financials.csv,✅ Complete
1,VCDB Cybersecurity Incidents,9911,25,vcdb_cybersecurity_incidents.csv,✅ Complete
2,BuiltWith Technology Stack,3,2,builtwith_technology_stack.csv,✅ Complete



📁 All output files saved to: /content/data/processed


### Step 4.2: Next Steps

## ✅ Next Steps for Your Analysis

1. **Data Cleaning**:
   - Handle missing values
   - Standardize company names across datasets
   - Convert date fields to datetime
   - Normalize numerical features

2. **Feature Engineering**:
   - Calculate financial ratios (debt-to-equity, profit margin, etc.)
   - Create binary breach indicator from VCDB
   - Aggregate technology counts from BuiltWith
   - Time-based features (years since founding, etc.)

3. **Data Merging**:
   - Match companies across datasets using:
     - Company name fuzzy matching
     - Domain name matching
     - CIK/ticker symbols
   
4. **Exploratory Data Analysis**:
   - Distribution of breaches by industry
   - Correlation between financial health and breach likelihood
   - Technology adoption patterns

5. **Modeling**:
   - Train/test split
   - Feature selection
   - Model training (Random Forest, XGBoost, etc.)
   - Cross-validation
   - Model evaluation

---

## 📚 Additional Resources

- **SEC EDGAR API Documentation**: https://www.sec.gov/edgar/sec-api-documentation
- **VCDB GitHub**: https://github.com/vz-risk/VCDB
- **BuiltWith API Docs**: https://api.builtwith.com/
- **Verizon DBIR Reports**: https://www.verizon.com/business/resources/reports/dbir/

---

## 🔧 Troubleshooting

**SEC Data Issues:**
- If rate limited, add longer delays between requests
- Ensure User-Agent header is set correctly

**VCDB Issues:**
- If git clone fails, manually download ZIP from GitHub
- Check that data path exists: `VCDB/data/json/validated`

**BuiltWith Issues:**
- Verify API key is correct
- Free tier is limited to 1 request/second
- Consider paid tier for production use

---